In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import random

In [27]:
def generate_currency_pair_df(path,pairname):
    pair_df = pd.read_csv(path,index_col = 0)
    pair_df['time'] = pd.to_datetime(pair_df['date'],unit='s')
    pair_df.drop(['date'],1,inplace=True)
    pair_df.time = pd.DatetimeIndex(pair_df.time) 
    pair_df = pair_df.set_index('time')
    pair_df.sort_index()
    hourly_pair_grouped = pair_df.groupby(pd.Grouper(freq='H'),) # after this grouping, becomes series
    hourly_pair_volume = hourly_pair_grouped.volume.sum()
    hourly_pair_qvolume = hourly_pair_grouped.quoteVolume.sum()
    hourly_pair_close = hourly_pair_grouped.close.mean()
    hourly_pair_open = hourly_pair_grouped.open.mean()
    hourly_pair_high = hourly_pair_grouped.high.max()
    hourly_pair_low = hourly_pair_grouped.low.min()
    hourly_weighted_mean = hourly_pair_grouped.apply(lambda dfx: (dfx["weightedAverage"] * dfx["volume"]).sum() / dfx["volume"].sum())
    hourly_weighted_mean = hourly_weighted_mean.rename(pairname+'_weighted_mean').reset_index().set_index('time')
    hourly_pair = pd.concat([hourly_pair_high, hourly_pair_low, hourly_pair_close, hourly_pair_open,hourly_pair_volume,hourly_pair_qvolume, hourly_weighted_mean],axis = 1)
    hourly_pair = hourly_pair.rename(columns={'high':pairname+'_high', 'low':pairname+'_low', \
                                            'volume':pairname+'_volume', 'close':pairname+'_close',\
                                             'open':pairname+'_open',  'quoteVolume':pairname+'_quoteVolume',})
    hourly_pair.replace([np.inf, -np.inf], np.nan)
    hourly_pair = hourly_pair.interpolate()
    hourly_pair = hourly_pair.dropna()
    hourly_pair[pairname+'_pctChange']=hourly_pair[pairname+'_weighted_mean'].pct_change()
    hourly_pair = hourly_pair.dropna()
    return hourly_pair

In [28]:
df1 = generate_currency_pair_df('../cs341/poloniex/USDT_BTC.csv','USDT_BTC')
df1.head()

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:15: RuntimeWarning: invalid value encountered in double_scalars
  from ipykernel import kernelapp as app


,USDT_BTC_high,USDT_BTC_low,USDT_BTC_close,USDT_BTC_open,USDT_BTC_volume,USDT_BTC_quoteVolume,USDT_BTC_weighted_mean,USDT_BTC_pctChange
time,,,,,,,,
2016-01-01 10:00:00,431.76,431.76,431.760000,431.760000,38.438328,0.089027,431.760000,0.001996
2016-01-01 11:00:00,432.62,431.76,432.548333,432.548333,16.616274,0.038485,431.765150,0.000012
2016-01-01 12:00:00,432.62,431.76,431.975000,431.975000,6.124615,0.014185,431.760000,-0.000012
2016-01-01 13:00:00,432.62,431.76,432.405000,432.405000,1849.916809,4.284399,431.779778,0.000046
2016-01-01 14:00:00,431.76,429.12,430.654167,430.655000,146.046809,0.338305,431.701385,-0.000182


In [22]:
nan_rows = df1[df1.isnull().T.any().T]
nan_rows

,USDT_BTC_high,USDT_BTC_low,USDT_BTC_close,USDT_BTC_open,USDT_BTC_volume,USDT_BTC_quoteVolume,USDT_BTC_weighted_mean,USDT_BTC_pctChange
time,,,,,,,,
2016-01-01 08:00:00,433.010137,433.010137,433.010137,433.010137,0.000000,0.000000,NaN,NaN
2016-01-01 09:00:00,433.010137,430.900000,431.251690,431.251690,26.826007,0.062256,430.900001,NaN


In [19]:
nan_rows.columns[nan_rows.isna().any()].tolist()

['USDT_BTC_weighted_mean', 'USDT_BTC_pctChange']

In [29]:
df2 = generate_currency_pair_df('../cs341/poloniex/USDT_ETH.csv','USDT_ETH')
df3 = generate_currency_pair_df('../cs341/poloniex/USDT_LTC.csv','USDT_LTC')
df4 = generate_currency_pair_df('../cs341/poloniex/USDT_XRP.csv','USDT_XRP')
df5 = generate_currency_pair_df('../cs341/poloniex/BTC_ETH.csv','BTC_ETH')
df6 = generate_currency_pair_df('../cs341/poloniex/BTC_LTC.csv','BTC_LTC')
df7 = generate_currency_pair_df('../cs341/poloniex/BTC_XRP.csv','BTC_XRP')


/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:15: RuntimeWarning: invalid value encountered in double_scalars
  from ipykernel import kernelapp as app


In [30]:
df_hourly_4_currency = df1.join(df2).join(df3).join(df4).join(df5).join(df6).join(df7)